# 1장 가설 검정

## 1. 상관분석

In [8]:
import pandas as pd
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
data = pd.DataFrame(diabetes['data'], columns=diabetes['feature_names'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
dtypes: float64(10)
memory usage: 34.7 KB


In [4]:
from scipy.stats import pearsonr
print(pearsonr(data['age'], data['bmi']))

PearsonRResult(statistic=0.18508466614655553, pvalue=9.076791865417336e-05)


In [5]:
print(data[['age', 'bmi']].corr())

          age       bmi
age  1.000000  0.185085
bmi  0.185085  1.000000


## 2. 정규성 검정

In [6]:
import numpy as np
np.random.seed(2024)
x = np.random.random(10)

In [7]:
from scipy.stats import shapiro
print(shapiro(x))

ShapiroResult(statistic=0.8969662710477171, pvalue=0.20285343366469372)


## 3. 모평균 검정

In [11]:
import numpy as np
from scipy.stats import ttest_1samp

kg = np.array([75.5, 83.9, 75.7, 56.2, 73.4, 67.7, 79.0,
               50.7, 58.4, 74.1, 65.1, 77.8, 48.1, 46.3])

print(kg.mean())

print(ttest_1samp(kg, 70))

66.56428571428572
TtestResult(statistic=-1.0289933120202257, pvalue=0.3222484823978743, df=13)


In [15]:
import pandas as pd
from scipy.stats import ttest_rel

female = np.array([50.7, 58.4, 74.1, 65.1, 77.8, 48.1, 46.3])
male = np.array([75.5, 83.9, 75.7, 56.2, 73.4, 67.7, 79.0])

diff = female - male
print(diff.mean())

print(ttest_rel(female, male))

-12.985714285714291
TtestResult(statistic=-2.078446933064972, pvalue=0.08291274205610201, df=6)


In [16]:
from scipy.stats import ttest_ind 

print(female.mean(), male.mean())

print(ttest_ind(female, male))

60.07142857142857 73.05714285714286
TtestResult(statistic=-2.2186641577772956, pvalue=0.046550122110569664, df=12.0)


## 4. 모분산 검정 (1)

In [22]:
import numpy as np
from scipy.stats import chi2
score = np.array([80.5, 60.2, 70, 87, 45, 91, 85])

var0 = 1100
var = np.var(score, ddof=1)
dof = len(score)-1

stat = (dof*var/var0)

print(chi2.cdf(stat, df=dof)) # 좌측 검정
print(1-chi2.cdf(stat, df=dof)) # 우측 검정
print(2*chi2.cdf(stat, df=dof)) # 양측 검정

0.041637780038918736
0.9583622199610813
0.08327556007783747


## 5. 모분산 검정 (2)

In [41]:
import numpy as np
from scipy.stats import f

a = np.array([70, 80, 75, 65, 100, 98])
b = np.array([20, 100, 50, 94, 28, 80, 95, 30])

# 가설 H0 : var_a = var_b, H1 : var_a < var_b
var_a = np.var(a, ddof=1)
var_b = np.var(b, ddof=1)
print(var_a, var_b)

df_a = len(a)-1
df_b = len(b)-1

stat = var_a / var_b
print(stat)

p_value = f.cdf(stat, df_a, df_b)
print(p_value)

212.66666666666669 1138.4107142857142
0.18681014154026346
0.041539430375629585


In [39]:
# var_b / var_a > 1

stat = var_b / var_a
1 - f.cdf(stat, df_b, df_a)

0.04153943037562957

## 6. 모분산 검정 (3)

In [2]:
from scipy.stats import bartlett
import numpy as np

a = np.array([70, 80, 75, 65, 100, 98])
b = np.array([20, 100, 50, 94, 28, 80])
c = np.array([90, 97, 95, 94, 99, 100])

print(bartlett(a, b, c))

BartlettResult(statistic=15.6702722148674, pvalue=0.00039558846873743075)


In [3]:
from scipy.stats import levene

print(levene(a, b, c))

LeveneResult(statistic=14.365736704446384, pvalue=0.00032713621045500125)


## 7. 분산분석

In [11]:
import pandas as pd
from statsmodels.formula.api import ols 
from statsmodels.stats.anova import anova_lm 

df = pd.read_csv('../data/예제/school_score.csv')
print(df)

one_way = ols('Final ~ C(School)', data = df).fit()

result = anova_lm(one_way)
print(result)

        id School Sex  Grade  Final
0        1      A   M      1   44.4
1        2      A   M      2   47.7
2        3      A   M      3   65.6
3        4      A   F      1   50.7
4        5      A   F      2   51.3
...    ...    ...  ..    ...    ...
2995  2996      C   M      2   84.1
2996  2997      C   M      3   81.6
2997  2998      C   F      1   83.2
2998  2999      C   F      2   92.8
2999  3000      C   F      3  100.0

[3000 rows x 5 columns]
               df         sum_sq        mean_sq            F  PR(>F)
C(School)     2.0  996939.237147  498469.618573  5722.221007     0.0
Residual   2997.0  261072.308290      87.111214          NaN     NaN


In [12]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey_result = pairwise_tukeyhsd(endog = df['Final'],
                                 groups = df['School'],
                                 alpha = 0.05)
print(tukey_result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     A      B  38.8464    0.0 37.8677 39.8251   True
     A      C  38.4922    0.0 37.5135 39.4709   True
     B      C  -0.3542 0.6728 -1.3329  0.6245  False
----------------------------------------------------


In [13]:
two_way = ols('Final ~ C(School) + C(Grade) + C(School):C(Grade)', data = df).fit()

result = anova_lm(two_way)
print(result)

                        df         sum_sq        mean_sq            F    PR(>F)
C(School)              2.0  996939.237147  498469.618573  5721.620556  0.000000
C(Grade)               2.0     324.472829     162.236415     1.862210  0.155509
C(School):C(Grade)     4.0     170.851257      42.712814     0.490274  0.742912
Residual            2991.0  260576.984204      87.120356          NaN       NaN


In [14]:
two_way = ols('Final ~ C(School) + C(Grade)', data = df).fit()

result = anova_lm(two_way)
print(result)

               df         sum_sq        mean_sq            F    PR(>F)
C(School)     2.0  996939.237147  498469.618573  5725.518315  0.000000
C(Grade)      2.0     324.472829     162.236415     1.863479  0.155312
Residual   2995.0  260747.835461      87.061047          NaN       NaN


## 8. 카이제곱 검정

In [20]:
from scipy.stats import chisquare

obs = [423, 304, 274, 205, 294]
exp_ratio = [1/5] * 5
exp = [i*sum(obs) for i in exp_ratio]

print(chisquare(obs, exp))

Power_divergenceResult(statistic=82.94, pvalue=4.14849046718008e-17)


In [26]:
import pandas as pd
from scipy.stats import chi2_contingency

crosstab = pd.DataFrame({'좋음' : [400, 350],
                         '싫음' : [950, 800]})

chi2, p_value, dof, exp_freq = chi2_contingency(crosstab)
print(f"검정통계량 : {chi2}")
print(f"유의확률 : {p_value}")
print(f"자유도 : {dof}")
print(f"기대도수 : {exp_freq}")

# 성별에 따라 선호도의 비율이 다르다고 할 수 없다

검정통계량 : 0.15527950310559005
유의확률 : 0.6935402833910855
자유도 : 1
기대도수 : [[405. 945.]
 [345. 805.]]


In [33]:
import pandas as pd
from scipy.stats import chi2_contingency

crosstab = pd.DataFrame({'Black' : [1620, 2380],
                         'Gold' : [385, 615],
                         'Purple' : [778, 1230],
                         'Red' : [394, 610],
                         'White' : [800, 188]})

chi2, p_value, dof, exp_freq = chi2_contingency(crosstab)

print(f"검정통계량 : {chi2}")
print(f"유의확률 : {p_value}")
print(f"자유도 : {dof}")
print(f"기대도수 : {exp_freq}")

# 성별과 색상은 서로 관련이 있다

검정통계량 : 611.2969703825839
유의확률 : 5.561479016921619e-131
자유도 : 4
기대도수 : [[1767.55555556  441.88888889  887.31288889  443.65644444  436.58622222]
 [2232.44444444  558.11111111 1120.68711111  560.34355556  551.41377778]]


## 9. 비모수 검정

### 9-1. 스피어만 상관계수 검정

In [46]:
from sklearn.datasets import load_diabetes
from scipy.stats import spearmanr

diabetes = load_diabetes()

# print(diabetes['feature_names'])
df = pd.DataFrame(diabetes['data'], columns = diabetes['feature_names'])

print(spearmanr(df['sex'], df['bmi']))

print(df[['sex', 'bmi']].corr(method='spearman'))

# 상관계수는 0.098로 작지만 유의확률이 0.05보다 작아 유의하다

SignificanceResult(statistic=0.09807947297621517, pvalue=0.03929011358104617)
          sex       bmi
sex  1.000000  0.098079
bmi  0.098079  1.000000


### 9-2. 켄달의 타우 검정

In [52]:
import numpy as np
from scipy.stats import kendalltau

x = np.array([5, 4, 3, 6, 1, 2])
y = np.array([1, 5, 2, 2, 2, 6])

print(kendalltau(x, y))

# 상관계수가 -0.298로 상관관계가 거의 없으며 유의확률이 0.42로 유의수준 0.05하에서 유의하지 않다

SignificanceResult(statistic=-0.29814239699997197, pvalue=0.4205962375999266)


### 9-3 윌콕슨 부호순위 검정

In [55]:
from scipy.stats import wilcoxon
import numpy as np

kg = np.array([75.5, 83.9, 75.7, 56.2, 73.4, 67.7, 79.0, 50.7, 58.4, 74.1, 65.1, 77.8, 48.1, 46.3])

print(wilcoxon(kg - 70))

# 검정통계량이 42.0이며 유의확률이 0.54로 유의수준 0.05 하에서 귀무가설을 기각 할 수 없다 (즉, 중위수가 70이라고 할 수 있다)

WilcoxonResult(statistic=42.0, pvalue=0.5416259765625)


In [57]:
import numpy as np
from scipy.stats import wilcoxon

female = np.array([50.7, 58.4, 74.1, 65.1, 77.8, 48.1, 46.3])
male = np.array([75.5, 83.9, 75.7, 56.2, 73.4, 67.7, 79.0])

diff = female - male

print(wilcoxon(diff))

# 검정통계량이 5.0이며 유의확률이 0.15로 유의수준 0.05 하에서 귀무가설을 기각 할 수 없다 (즉, 두 표본의 중앙값에 차이가 없다)

WilcoxonResult(statistic=5.0, pvalue=0.15625)


### 9-4 윌콕슨 순위합 검정

In [58]:
from scipy.stats import ranksums 

print(ranksums(female, male))

RanksumsResult(statistic=-1.8527420384998257, pvalue=0.06391934147515746)


### 9-5 만-휘트니 U 검정

In [59]:
from scipy.stats import mannwhitneyu 

print(mannwhitneyu(female, male))

MannwhitneyuResult(statistic=10.0, pvalue=0.07284382284382285)


### 9-6 크루스칼 왈리스 H 검정

In [8]:
import pandas as pd
from scipy.stats import kruskal
df = pd.read_csv('../data/예제/school_score.csv')

School = df['School']
Final = df['Final']

Final_A = df[df['School'] == 'A']['Final']
Final_B = df[df['School'] == 'B']['Final']
Final_C = df[df['School'] == 'C']['Final']

kruskal(Final_A, Final_B, Final_C)

KruskalResult(statistic=1978.1734792103248, pvalue=0.0)

# 2장 통계 모형

## 1. 선형 회귀모형

### 1-1. 단순 선형 회귀분석

In [7]:
import pandas as pd
from sklearn.datasets import load_diabetes 
from scipy.stats import linregress

diabetes = load_diabetes()
# print(diabetes.keys())


df = pd.DataFrame(diabetes['data'], columns = diabetes['feature_names'])
# print(df)

model = linregress(x = df['bmi'], y = diabetes['target'])
print(model)

LinregressResult(slope=949.4352603840387, intercept=152.13348416289617, rvalue=0.5864501344746886, pvalue=3.466006445167251e-42, stderr=62.51512200285265, intercept_stderr=2.9735411187907346)


In [13]:
# beta1
print(model.slope)

# beta0
print(model.intercept)

# 회귀식 : target = 152.13 + 949.43 * bmi

949.4352603840387
152.13348416289617


In [15]:
# beta1에 대한 통계적 유의성
print(model.pvalue)

# 결정계수
print(model.rvalue)

3.466006445167251e-42
0.5864501344746886


### 1-2. 다중 선형 회귀분석

In [19]:
import pandas as pd
import statsmodels.api as sm
tips = pd.read_csv('../data/예제/tips.csv')
# print(tips)

X = tips[['total_bill', 'size']]
y = tips['tip']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    tip   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     105.9
Date:                Wed, 30 Oct 2024   Prob (F-statistic):           9.67e-34
Time:                        10:23:21   Log-Likelihood:                -347.99
No. Observations:                 244   AIC:                             702.0
Df Residuals:                     241   BIC:                             712.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6689      0.194      3.455      0.0

In [45]:
print(X.iloc[4])

print(model.predict(X.iloc[4]))

const          1.00
total_bill    24.59
size           4.00
Name: 4, dtype: float64
None    3.719157
dtype: float64


In [58]:
new_data = pd.DataFrame({
    'const' : [1.0],
    'total_bill' : [24.59],
    'size' : [4]
})

result = model.get_prediction(new_data)
print(result.summary_frame())

       mean  mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  obs_ci_upper
0  3.719157  0.12093       3.480943       3.957371      1.708534      5.729779


## 2절 로지스틱 회귀모형

In [61]:
import pandas as pd
import statsmodels.api as sm

survived = pd.read_csv('../data/예제/survived.csv')
# print(survived)

X = survived['pclass']
y = survived['survived']

X = sm.add_constant(X)
# print(X)

model = sm.GLM(y, X, family = sm.families.Binomial()).fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               survived   No. Observations:                  541
Model:                            GLM   Df Residuals:                      539
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -315.13
Date:                Wed, 30 Oct 2024   Deviance:                       630.26
Time:                        11:36:13   Pearson chi2:                     541.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1527
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.1558      0.124     -9.293      0.0

In [62]:
import numpy as np

# 로지스틱 회귀모형 : logit(y) = -1.1558 + 1.8009 * pclass

Prob_A = np.exp(-1.1558+1.8009*1) / (1+np.exp(-1.1558+1.8009*1))
Prob_B = np.exp(-1.1558+1.8009*0) / (1+np.exp(-1.1558+1.8009*0))

print(Prob_A, Prob_B)

0.6559054109099537 0.2394312844887346


In [66]:
# 적합모형의 이탈도
dev = model.deviance
print(dev)

# 영모형의 이탈도
dev0 = model.null_deviance
print(dev0)

stat = dev0 - dev
df = 2 - 1
print(stat, df)

from scipy.stats import chi2
pval = 1 - chi2.cdf(stat, df)
print(pval)   

630.2646521014273
719.8918959915943
89.62724389016705 1
0.0
